# Social Media

> Help me with social media

In [ ]:
#| default_exp social_media

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from random import randint
from fastlite.core import *
from base64 import b64encode
from fractionalindex.fractionalindex import SqliteIndex
from utils import *

## Database

We have two tables.
+ `posts` is made up of `Post`s, each corresponds to a single twitter post.  This is made to render posts and determine the `position` in the thread.
+ `threads` is made up of `Thread`s, which is a collection of posts.  This is used to have a human readable name assigned and identify the user the thread belogns to

In [ ]:
#| export
ar = APIRouter(prefix='/social_media', body_wrap=layout)

In [ ]:
#| export
# This is from Alexis' fractional indexing which will help us manipulate ordering of cells
idx = SqliteIndex(db.conn,'posts', col='position')

In [ ]:
#| export
def FooterIcon(icon, value=None):
    "Icons that look like twitter UI but do absolutely nothing"
    return Button(UkIcon(icon), Span(value) if value else None, cls=ButtonT.ghost) 

In [ ]:
#| export
def PostControls(pid, tid):
    "Buttons that allow users to make actions, such as add/remove images, or reorder posts"
    return DivFullySpaced(
        Div(Button("Emoji", uk_toggle=f'#emoji_picker-{pid}'),
            Div(EmojiPicker(pid, tid), id=f'emoji_picker-{pid}', hidden=True, cls='absolute w-[400px]')),
        Div(Button("Add image", uk_toggle=f'#upload-modal-{pid}'),
            Div(UploadZone(pid,tid), id=f'upload_zone-{pid}', hidden=True, cls='absolute')),
        Button("Remove Image", hx_delete='rm_image', hx_target=f'#img-{pid}', hx_swap='innerHTML', hx_vals={'pid':pid}),
        Button("New Post",     hx_get=TwitterPost.to(tid=tid),   hx_target=f'#post-list', hx_swap='beforeend'), 
        Button(UkIcon("move-up"),   hx_trigger='click', hx_target='#post-list', hx_get=mv_post_up.to(pid=pid,tid=tid)),
        Button(UkIcon("move-down"), hx_trigger='click', hx_target='#post-list', hx_get=mv_post_down.to(pid=pid, tid=tid)))

In [ ]:
#| export
@ar
def PostTextArea(txt:str, pid:str, tid:str):
    "The actual text content is editable here, and includes a counter for how many chars it has."
    txt = txt if txt else ''
    return Div(
        TextArea(txt,
                name='content',
                id=f'content-{pid}',
                cls='w-full border-0 focus:outline-none resize-none bg-transparent',
                hx_post=save_post_content,
                hx_trigger='keyup delay:100ms changed',
                hx_vals={'pid':pid,'tid':tid},
                oninput=f"""
                    var count = this.value.length;
                    var countEl = document.getElementById('char-count-{pid}');
                    countEl.textContent = count + '/280';
                    countEl.className = count > 280 ? 'text-sm text-red-500 text-right mt-1' : 'text-sm text-gray-500 text-right mt-1';
                """),
        Div(f"{len(txt)}/280", 
            id=f"char-count-{pid}",
            cls=f"text-sm {'text-red-500' if len(txt)>280 else 'text-gray-500'} text-right mt-1"),
        id=f'content-container-{pid}')

In [ ]:
#| export
@ar
def mv_post_down(pid:str, tid:str, sess): 
    "Use Fractional Indexing to move post down in the thread"
    curr_pos = db.fetchone('select position from posts where id=?', (pid,))
    _position = db.fetchone('select min(position) as m from posts where position>? and tid=?', [curr_pos, tid])
    post_tbl.upsert(Post(id=pid, position=idx.insert(after=_position)))
    return load_thread(tid, sess['user_name'])
@ar
def mv_post_up(pid:str, tid:str, sess): 
    "Use Fractional Indexing to move post down in the thread"
    curr_pos = db.fetchone('select position from posts where id=?', (pid,))
    _position = db.fetchone('select max(position) as m from posts where position<? and tid=?', [curr_pos, tid])
    post_tbl.upsert(Post(id=pid, position=idx.insert(before=_position)))
    
    return load_thread(tid, sess['user_name'])

In [ ]:
#| export
def autosize_textarea():
    return Script("""
    document.querySelectorAll('textarea').forEach(textarea => {
        textarea.style.height = 'auto';
        textarea.style.height = textarea.scrollHeight + 'px';
        textarea.addEventListener('input', e => {
            e.target.style.height = 'auto';
            e.target.style.height = e.target.scrollHeight + 'px';});});""")

In [ ]:
#| export
emojis = [
    "😀", "😃", "😄", "😁", "😅", "😂", "🤣", "😊", "😇", "🙂", "🙃", "😉", "😌", "😍", "🥰", 
    "😘", "😗", "😙", "😚", "😋", "😛", "😝", "😜", "🤪", "🤨", "🧐", "🤓", "😎", "🤩", "🥳",
    "😏", "😒", "😞", "😔", "😟", "😕", "🙁", "☹️", "😣", "😖", "😫", "😩", "🥺", "😢", "😭",
    "❤️", "🧡", "💛", "💚", "💙", "💜", "🖤", "🤍", "🤎", "💔", "❤️‍🔥", "❤️‍🩹", "❣️", "💕", "💞",
    "🌟", "⭐", "✨", "💫", "🌙", "☀️", "⚡", "🔥", "💥", "❄️", "🌈", "🌊", "🍀", "🌸", "🌺"
]

def EmojiButton(pid, tid, emoji):
    "A button that shows and emoji that when pressed will append the emoji to the end of the text area"
    return Button(emoji, cls=ButtonT.ghost, hx_post=insert_emoji,
        hx_vals=f'''js:{{pid: '{pid}', tid: '{tid}', emoji: '{emoji}', content: document.querySelector('#content-{pid}').value}}''',
        hx_target=f'#content-container-{pid}', hx_swap='outerHTML', uk_toggle=f'#emoji_picker-{pid}')

In [ ]:
#| export
def EmojiPicker(pid, tid):
    "A grid of emoji buttons users can select from"
    return Card(
        Grid(*(EmojiButton(pid, tid, emoji) for emoji in emojis), cls='', cols=10),cls='w-fit z-50 uk-background-muted')

In [ ]:
#| export
@ar 
def insert_emoji(pid:str, tid:str, emoji:str, content:str):
    "Appends an emoji to content and returns an updated PostTextArea"
    new_content = content + emoji
    save_post_content(pid, tid, new_content)
    return PostTextArea(new_content, pid, tid)

In [ ]:
#| export
def UploadZone(pid, tid, icon="upload", text="Drag files here or click to upload"):
    """Creates a styled upload zone modal for users to drag/drop images into using MonsterUI modal"""
    return Modal(DivCentered(
            UkIcon(icon, cls=TextT.muted),
            P(text, cls=TextPresets.muted_lg),
            Input(type="file", name="image", 
                cls="opacity-0 absolute inset-0 w-full h-full cursor-pointer",
                hx_post=upload_image, 
                hx_encoding="multipart/form-data", 
                hx_trigger="change", 
                hx_vals={'pid':pid,'tid':tid},
                hx_target=f'#img-{pid}'),
        cls='space-y-10'),
              id=f'upload-modal-{pid}')

@ar
async def upload_image(pid:str, tid:str, image:UploadFile):
    "Save image to disk and insert image into db/UI"
    contents = await image.read()
    img_data = f"data:{image.content_type};base64,{b64encode(contents).decode()}"
    os.makedirs(f"uploads/{tid}/{pid}", exist_ok=True)
    filename = f"uploads/{tid}/{pid}/{image.filename}"
    Path(filename).write_bytes(contents)
    save_post_image(pid, tid, f"/{filename}")
    return Img(src=f"/{filename}", cls="rounded-lg max-w-[512px] mt-4"), UploadZone(pid, tid) 

In [ ]:
#| export
@ar
def save_post_content(pid:str, tid:str, content:str): 
    if pid in post_tbl: post_tbl.upsert(Post(id=pid, tid=tid, content=content))
    else: post_tbl.insert(Post(id=pid, tid=tid, position=idx.insert()))
def save_post_image(pid:str, tid:str, img_path:str):  post_tbl.upsert(Post(id=pid, tid=tid, img_path=img_path))
    
@ar.delete
def rm_image(pid:str):post_tbl.upsert(Post(id=pid, img_path=None))

In [ ]:
#| export
@ar
def TwitterPost(tid:str, pid=None, txt='', img=None, username="User", handle="@user", time="Just now"):
    "Creates the UI look for a Twitter Post"
    if not pid: pid = unqid()
    return Div(
        Card(
            DivLAligned(
                Span(username, cls=TextT.bold),
                *map(lambda x: Span(x,cls=TextPresets.muted_sm), (handle,time))),
            PostTextArea(txt,pid, tid),
            autosize_textarea(),
            Div(Img(src=img,cls='rounded-lg max-w-[512px]') if img else None, id=f'img-{pid}'),
            DivFullySpaced(
                *[FooterIcon(i,v) for i,v in (('chat','0'), ('repeat', randint(5, 100)), ('heart', randint(100, 900)))],
                UkIcon('share'),
                cls=TextT.muted),
            cls='shadow-none'),PostControls(pid,tid))

In [ ]:
#|export
@ar.delete
def rm_thread(tid:str, sess):
    "Deletes a thread and all it's posts"
    #TODO:  Delete the images from disk too
    user_name = db.fetchone('select user from threads where id=?', (tid,))
    assert sess['user_name'] == user_name
    thread_tbl.delete(tid)
    post_tbl.delete_where('tid=?', (tid,))
    return Div(id='post-list', cls='space-y-5'), mk_thread_list(sess['user_name'])

In [ ]:
#| export
@ar
def load_thread(tid:str, user_name:str):
    "Loads a thread from DB with header with title/delete, a list of `TwitterPost`s, and the thread list selector"
    posts = list(post_tbl.rows_where('tid=?',(tid,), order_by='position'))
    thread_name = db.fetchone('select name from threads where id=?', (tid,))

    return Div(DivFullySpaced(H3(thread_name),Button("Delete", cls=ButtonT.danger, hx_delete=rm_thread, hx_target='#post-list', hx_swap='outerHTML', hx_vals={'tid':tid}, hx_confirm='Are you sure?  The thread and all posts will be deleted')),
            *(TwitterPost(
        tid=p['tid'],
        pid=p['id'],
        txt=p['content'],
        img=p['img_path'],
        username=p['username'],
        handle=p['handle'],
        time=p['created_at']
    ) for p in posts), id='post-list', cls='space-y-5 max-w-[598px]'), mk_thread_list(user_name, tid)

In [ ]:
#| export
@ar
def mk_thread(sess, thread_name:str):
    "Create a new twitter thread in db, load it into UI, then update the thread list in UI"
    pid, tid = unqid(), unqid()
    post_tbl.upsert(Post(id=pid, tid=tid, position=idx.insert()))
    thread_tbl.upsert(Thread(id=tid, name=thread_name, user=sess['user_name']))
    return Div(load_thread(tid, sess['user_name'])),mk_thread_list(sess['user_name'], tid)

In [ ]:
#| export
def mk_thread_list(user_name, tid=None):
    "Creates UI for a list of all threads in the db for the given user"
    threads = list(thread_tbl.rows_where('user=?', (user_name,)))
    threadlist = [Li(A(thread['name']), hx_get=load_thread.to(tid=thread['id'], user_name=user_name), hx_target='#post-list', cls='uk-active' if tid==thread['id'] else '') for thread in threads]
    return NavContainer(id='thread-list', hx_swap_oob='true', cls=(NavT.primary,'h-full'))(*threadlist)

In [ ]:
#| export
@ar
def index(sess):
    return Title("PostHaste"), Container(H3("Make your Twitter thread post haste!", cls='mb-10'),
        Grid(
            Div(Form(DivFullySpaced(
                    Input(id="thread_name", name='thread_name', placeholder='Thread Name'),
                    Button("Create", cls=ButtonT.primary),
            cls='mb-5 space-x-3'), 
                    hx_get=mk_thread, hx_target='#post-list', hx_swap='outerHTML'),
                Div(mk_thread_list(sess['user_name']), id='thread-list')),
            Div(id='post-list', cls='space-y-3 max-w-[598px]'),))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()